In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, PReLU

In [4]:
df = pd.read_csv('30_emails_en.csv')
df.head()

,index,email_from,data,label
0,0,['lehoangminh.ai.tech@gmail.com'],Unprecedented shocking promotion: shop without...,spam
1,1,['lehoangminh.ai.tech@gmail.com'],Get 5 million VND instantly with just a few si...,spam
2,2,['lehoangminh.ai.tech@gmail.com'],You have won the special prize from our promot...,spam
3,3,['lehoangminh.ai.tech@gmail.com'],Secret information just for you: a huge cash o...,spam
4,4,['lehoangminh.ai.tech@gmail.com'],Click the link to receive your prize instantly...,spam


In [5]:
label_encoder = LabelEncoder()
df['label_encode'] = label_encoder.fit_transform(df['label'])
print(label_encoder.classes_) # tương đương các nhãn sau khi labelencode

['advertising' 'entertainment' 'friends' 'spam' 'study' 'work']


In [6]:
df.head()

,index,email_from,data,label,label_encode
0,0,['lehoangminh.ai.tech@gmail.com'],Unprecedented shocking promotion: shop without...,spam,3
1,1,['lehoangminh.ai.tech@gmail.com'],Get 5 million VND instantly with just a few si...,spam,3
2,2,['lehoangminh.ai.tech@gmail.com'],You have won the special prize from our promot...,spam,3
3,3,['lehoangminh.ai.tech@gmail.com'],Secret information just for you: a huge cash o...,spam,3
4,4,['lehoangminh.ai.tech@gmail.com'],Click the link to receive your prize instantly...,spam,3


In [7]:
x_train, x_test, y_train, y_test = [], [], [], []
for label in label_encoder.classes_:
    df_optional = df[df['label'] == label]
    x = df_optional['data']
    y = df_optional['label_encode']
    x_1, x_2, y_1, y_2 = train_test_split(x, y, test_size=0.2)
    x_train.append(x_1)
    x_test.append(x_2)
    y_train.append(y_1)
    y_test.append(y_2)

# print(f'X train: {x_train}')
# print(f'X test: {x_test}')
# print(f'Y train: {y_train}')
# print(f'Y test: {y_test}')



In [ ]:
# from gensim.models import Word2Vec

# # Tách từng câu thành danh sách các từ
# sentences = [text.split() for text in df['data']]

# # Huấn luyện mô hình Word2Vec
# w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Chuyển mỗi văn bản thành vector trung bình các từ
# def text_to_w2v(text, model, vector_size):
#     words = text.split()
#     word_vecs = [model.wv[w] for w in words if w in model.wv]
#     if word_vecs:
#         return np.mean(word_vecs, axis=0)
#     else:
#         return np.zeros(vector_size)

# X_w2v = np.array([text_to_w2v(text, w2v_model, 100) for text in df['data']])


In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score, recall_score

# # Predict the labels for the input data X
# predictions = model.predict(X)
# predicted_classes = predictions.argmax(axis=1)

# # Convert one-hot encoded labels to class indices
# true_classes = labels.argmax(axis=1)

# result_df = pd.DataFrame({
#     'Original Label': true_classes,
#     'Predicted Label': predicted_classes
# })

# # Calculate precision and recall
# precision = precision_score(true_classes, predicted_classes, average='macro')
# recall = recall_score(true_classes, predicted_classes, average='macro')

# print(f"Precision (macro): {precision:.2f}")
# print(f"Recall (macro): {recall:.2f}")
# # Calculate accuracy
# accuracy = accuracy_score(true_classes, predicted_classes)
# print(f"Model accuracy on the dataset: {accuracy:.2f}")

NameError: name 'model' is not defined

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Load tokenizer & model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Spam/Not spam

# Ví dụ email
emails = df['data']
labels = df['label_encode']  # 1 = Spam, 0 = Not spam

# Tokenize
inputs = tokenizer(emails, padding=True, truncation=True, return_tensors="pt")

# Forward
# outputs = model(**inputs, labels=torch.tensor(labels))
# Chuẩn bị dataset cho Trainer

class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo dataset
train_dataset = EmailDataset(inputs, labels.tolist())

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=True  
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Train
trainer.train()
loss = outputs.loss
print(loss, logits)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


int32


In [92]:
# Test nhiều sample mới kèm cả label thật để so sánh
sample_list = [
    ("Congratulations! You have been selected to receive a $1000 gift card. Click the link below to claim your prize now. Hurry, this offer expires soon!", "spam"),
    ("Huge weekend sale: 50% off all items. Visit our store today to grab the best deals!", "advertising"),
    ("New action movie released this week with amazing reviews. Don't miss it!", "entertainment"),
    ("Hey, let's meet this weekend for coffee and catch up.", "friends"),
    ("The final exam schedule has been posted. Please check the portal and prepare accordingly.", "study"),
    ("Reminder: project meeting at 9am tomorrow in the main conference room.", "work")
]

texts = [t for t, _ in sample_list]
true_labels = [lbl for _, lbl in sample_list]

# Tokenize and pad
sample_sequences = tokenizer.texts_to_sequences(texts)
X_samples = pad_sequences(sample_sequences, maxlen=maxlen)

# Predict
sample_preds = model.predict(X_samples)
pred_idxs = sample_preds.argmax(axis=1)
pred_labels = label_encoder.classes_[pred_idxs]
pred_probs = sample_preds[np.arange(len(sample_preds)), pred_idxs]

# Build result DataFrame including probability per class
probs_df = pd.DataFrame(sample_preds, columns=label_encoder.classes_)
result_samples_df = pd.DataFrame({
    "text": texts,
    "true_label": true_labels,
    "predicted_label": pred_labels,
    "predicted_prob": pred_probs
})
result_samples_df = pd.concat([result_samples_df, probs_df], axis=1)
print(result_samples_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
                                                text     true_label  \
0  Congratulations! You have been selected to rec...           spam   
1  Huge weekend sale: 50% off all items. Visit ou...    advertising   
2  New action movie released this week with amazi...  entertainment   
3  Hey, let's meet this weekend for coffee and ca...        friends   
4  The final exam schedule has been posted. Pleas...          study   
5  Reminder: project meeting at 9am tomorrow in t...           work   

  predicted_label  predicted_prob  advertising  entertainment   friends  \
0           study        0.340628     0.093331       0.147943  0.113268   
1         friends        0.338270     0.132916       0.110144  0.338270   
2         friends        0.310360     0.113462       0.111156  0.310360   
3         friends        0.507596     0.090963       0.073900  0.507596   
4         friends        0.568162     0.093068       0.064383  0.568162   
5         fri

In [93]:
result_samples_df

,text,true_label,predicted_label,predicted_prob,advertising,entertainment,friends,spam,study,work
0,Congratulations! You have been selected to rec...,spam,study,0.340628,0.093331,0.147943,0.113268,0.236107,0.340628,0.068723
1,Huge weekend sale: 50% off all items. Visit ou...,advertising,friends,0.338270,0.132916,0.110144,0.338270,0.107742,0.180722,0.130205
2,New action movie released this week with amazi...,entertainment,friends,0.310360,0.113462,0.111156,0.310360,0.098051,0.232162,0.134809
3,"Hey, let's meet this weekend for coffee and ca...",friends,friends,0.507596,0.090963,0.073900,0.507596,0.064984,0.152078,0.110480
4,The final exam schedule has been posted. Pleas...,study,friends,0.568162,0.093068,0.064383,0.568162,0.065134,0.115551,0.093702
5,Reminder: project meeting at 9am tomorrow in t...,work,friends,0.492598,0.092463,0.078325,0.492598,0.068180,0.156767,0.111667
